In [2]:
import numpy as np
import cv2
#from matplotlib import pyplot as plt
#from torchvision import datasets ,models,transforms
import os
#import matplotlib.image as img

#from torch.utils.data import Dataset, DataLoader
#from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
#from torch.optim import Adam
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#from torch.nn.modules.module import Module

import pickle
import math
#import app
import tqdm
from tqdm import trange

In [4]:
# CPU or GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
device='cpu'

# Data

In [3]:
#讀取train、validation、test
f = open("../train.txt")
train_txt = f.readlines()
f.close()
f = open("../val.txt")
val_txt = f.readlines()
f.close()
f = open("../test.txt")
test_txt = f.readlines()
f.close()

In [4]:
def readpic(p):
    #return mpimg.imread(p)
    return cv2.imread(p)

def showpic(pic):
    plt.imshow(pic)
    plt.show()

def more_showpic(pic_list):
    fig,ax = plt.subplots(1,5,figsize = (15,3))
    for idx, vactor in enumerate(pic_list[:5]):
        #ax[idx].imshow(vactor)
        ax[idx].imshow(cv2.cvtColor(vactor, cv2.COLOR_BGR2RGB))

In [5]:
#train image
train_img_path = []
train_img_label = []
for img_path in train_txt:
    train_path , train_label = img_path.split( )
    train_img_path.append(train_path)
    train_img_label.append(train_label)
#validation image   
val_img_path = []
val_img_label = []
for img_path in val_txt:
    val_path , val_label = img_path.split( )
    val_img_path.append(val_path)
    val_img_label.append(val_label)
#test image     
test_img_path = []
test_img_label = []
for img_path in test_txt:
    test_path , test_label = img_path.split( )
    test_img_path.append(test_path)
    test_img_label.append(test_label)

In [6]:
train = []
val = []
test = []
i = 0
for train_path in train_img_path:
    train.append(readpic('../'+train_path))
        
for val_path, test_path in zip(val_img_path, test_img_path):
    val.append(readpic('../'+val_path))
    test.append(readpic('../'+test_path))

In [9]:
X_train = np.array(train)
X_val = np.array(val)
X_test = np.array(test)

X_train, X_val, X_test = X_train/float(255), X_val/float(255), X_test/float(255)
# X_train -= np.mean(X_train)
# X_val -= np.mean(X_val)
# X_test -= np.mean(X_test)


/home/re6091054/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/home/re6091054/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/re6091054/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

ValueError: operands could not be broadcast together with shapes (256,343,3) (341,256,3) 

In [30]:
def resize_image(X, resized_list):
    for i in trange(len(X)):
        resized_X = cv2.resize(X[i], (64, 64), interpolation=cv2.INTER_AREA)
        resized_list.append(resized_X)

In [25]:
resized_X_train = []
for i in trange(len(X_train)):
    resized_X_train0 = cv2.resize(X_train[i], (64, 64), interpolation=cv2.INTER_AREA)
    resized_X_train.append(resized_X_train0)

100%|██████████| 63325/63325 [02:00<00:00, 527.67it/s] 


In [31]:
resized_X_val = []
resize_image(X_val, resized_X_val)

100%|██████████| 450/450 [00:00<00:00, 551.07it/s]


In [33]:
resized_X_test = []
resize_image(X_test, resized_X_test)

100%|██████████| 450/450 [00:01<00:00, 333.01it/s]


In [36]:
Y_train = np.array(train_img_label).astype('int64')
Y_val = np.array(val_img_label).astype('int64')
Y_test = np.array(test_img_label).astype('int64')

In [1]:
Y_val

NameError: name 'Y_val' is not defined

In [37]:
train = {'train_data':resized_X_train, 'train_label':Y_train}
valid = {'valid_data':resized_X_val, 'valid_label':Y_val}
test = {'test_data':resized_X_test, 'test_label':Y_test}

save data

In [38]:
with open('../data_pickle/resized_train_data.pickle', 'wb') as f:
    pickle.dump(train, f)
with open('../data_pickle/resized_valid_data.pickle', 'wb') as f:
    pickle.dump(valid, f)
with open('../data_pickle/resized_test_data.pickle', 'wb') as f:
    pickle.dump(test, f)

load data

In [39]:
with open('../data_pickle/resized_train_data.pickle', 'rb') as f :
    train = pickle.load(f)
with open('../data_pickle/resized_valid_data.pickle', 'rb') as f :
    valid = pickle.load(f)
with open('../data_pickle/resized_test_data.pickle', 'rb') as f :
    test = pickle.load(f)

In [40]:
X_train = train['train_data']
Y_train = train['train_label']
X_val = valid['valid_data']
Y_val = valid['valid_label']
X_test = test['test_data']
Y_test = test['test_label']

In [41]:
len(X_train),X_train[0].shape

(63325, (64, 64, 3))